<a href="https://colab.research.google.com/github/joplaete/faceMLtools/blob/main/Face_detect_recog_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python
!pip install pillow

!pip install face-recognition

!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt2.xml

     |████████████████████████████████| 100.1 MB 1.1 MB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=59f8e3bd31bf926c3116497c17c72d9bfafd752360aadf94de77a6626387d5ef
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models
--2022-01-18 22:16:10--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt2.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 540616 (528K) [text/plain]
Saving to: ‘haarcascade_frontalface_alt2.xml’

haarcascade_frontal 100%[===================>] 527.95K  --.-KB/s    in 0.02s   

2022-01-18 22:16:10 (34.1 MB/s) -

In [2]:
# make sure you got a gpu runtime!
!nvidia-smi

Tue Jan 18 22:16:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#@title Connect your drive that hosts your files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#@title Imports and Supporting Function, run once
import face_recognition
import cv2
from PIL import Image
import os, time
import numpy as np

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation = inter)
   return resized

In [7]:
#@title Compute encodings for Face Recognition

# images of person to find (helps to align them first)
person_to_find_images = r"/content/drive/MyDrive/FACE_RECOG_EXTRACT/PERSON_TO_FIND/daniel_craig/aligned" # @param str
# encodings get saved and re-used if found in folder, unless force_recompute is on
force_recompute = True #@param {type:"boolean"}

encodings_path = os.path.join(person_to_find_images, 'encodings.npy')
encodings = []
if not force_recompute and os.path.exists(encodings_path):
    print('Found and using existing encoding: ', encodings_path)
    print('(delete this or enabled force_recompute to recompute when image set has changed)')
    encodings = np.load(encodings_path)
else:
    print('Build face encodings')
    for im in os.listdir(person_to_find_images):
        if not 'jpg' in im: continue
        known_image = face_recognition.load_image_file(os.path.join(person_to_find_images, im))
        encoding = face_recognition.face_encodings(known_image)
        if len(encoding):
            print('-->> Adding: ', im)
            encodings.append(encoding[0])
np.save(encodings_path, encodings)

Build face encodings
-->> Adding:  000047.jpg.jpg
-->> Adding:  000528.jpg.jpg
-->> Adding:  daniel_craig_0002.jpg.jpg
-->> Adding:  daniel_craig_0001.jpg.jpg
-->> Adding:  daniel_craig_0003.jpg.jpg
-->> Adding:  daniel_craig_0004.jpg.jpg
-->> Adding:  daniel_craig_0005.jpg.jpg
-->> Adding:  daniel_craig_0007.jpg.jpg
-->> Adding:  daniel_craig_0011.jpg.jpg
-->> Adding:  daniel_craig_0013.jpg.jpg
-->> Adding:  daniel_craig_0012.jpg.jpg
-->> Adding:  daniel_craig_0015.jpg.jpg
-->> Adding:  daniel_craig_0018.jpg.jpg
-->> Adding:  daniel_craig_0019.jpg.jpg
-->> Adding:  daniel_craig_0020.jpg.jpg
-->> Adding:  daniel_craig_0021.jpg.jpg
-->> Adding:  daniel_craig_0022.jpg.jpg
-->> Adding:  daniel_craig_0024.jpg.jpg
-->> Adding:  daniel_craig_0025.jpg.jpg
-->> Adding:  interview1.jpg
-->> Adding:  interview1_01.jpg
-->> Adding:  interview2_01.jpg
-->> Adding:  interview2_02.jpg
-->> Adding:  interview2_03.jpg
-->> Adding:  interview3_01.jpg
-->> Adding:  interview3_02.jpg


In [9]:
#@title Find and Extract Recognized faces

mp4path = r"/content/drive/MyDrive/FACE_RECOG_EXTRACT/MEDIA_TO_SEARCH/Daniel Craig James Bond Monologue - SNL.mp4" # @param str

# OUT    
out = r"/content/drive/MyDrive/FACE_RECOG_EXTRACT/OUT/daniel_craig__late_night_show" # @param str
out_prexix = r'out.' # @param str
if not os.path.exists(out):
    os.mkdir(out)

# use 1 not 0 for start 
start_offset =  500# @param int
limit_frames =-1 # @param int # handy for testing, will only compute limit_frames
vidcap =  cv2.VideoCapture(mp4path)
vidcap.set(cv2.CAP_PROP_POS_FRAMES, start_offset-1)
print('cv2.CAP_PROP_POS_FRAMES:',cv2.CAP_PROP_POS_FRAMES)

print('\n START PROCESSING', vidcap)
print(mp4path)
print(out)
print('\n\n')

success, image = vidcap.read()
frame_count = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
face_cascade = cv2.CascadeClassifier(r"/content/haarcascade_frontalface_alt2.xml")

count = start_offset-1
while success:
    # cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
    # if count < clamp_start or count > clamp_end:
    #     count+=1
    #     print('skip', count)
    #     continue
    print('\nProcess frame: ', count, '/', int(frame_count), '  --  ', round(count/frame_count*100), '%')
    # print(mp4path, out)
    
    success, image = vidcap.read()
    if not success:
        print('-- FRAME READ FAILED, aborting.')
        break

    imshowname = os.path.split(mp4path)[1] + ' (RESIZED for PREVIEW)'
    # cv2.imshow(imshowname, image_resize(image, height = 800))
    # cv2.waitKey(1)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # # Detect faces using opencv
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    print('>> Found', len(faces),'face(s)')
    if len(faces) > 6: continue
    padding = 150 # @param float # padding for extraction
    min_face_size =  60# @param float # skip w or h smaller than this
    for i, (x, y, w, h) in enumerate(faces):
        print('------ Face', i)
        if h < min_face_size: print(f'too small ({min_face_size})'); continue
        if w < min_face_size: print(f'too small ({min_face_size})'); continue
        preview = image
        # preview = cv2.rectangle(image, (x-padding,y-padding), (x+w+padding,y+h+padding), (200,200,200), 15)
        # cv2.imshow(imshowname, image_resize(preview, height = 800))
        # cv2.waitKey(1)
        # cv2.rectangle(image, (x-padding, y-padding), (x+w+padding, y+h+padding),
                    # (0, 0, 255), 2)
        # print('Face: ',x,y,w,h)
        face = image[y-padding:y + h+padding, x-padding:x + w+padding]
        # # face_grey = gray[y-padding:y + h+padding, x-padding:x + w+padding]

        # protect against dodgy images
        if face.shape[0]==0 or face.shape[1]==0: continue
        # cv2.imshow("face", face)
        # cv2.waitKey(0) 
        # cv2.destroyAllWindows() 

        # print('**** comparing --', x,y,w,h)
        unknown_image = face#face_recognition.load_image_file(r"C:\Users\jopla\Projects\face_extractor\datasets\daniel_craig\daniel_craig_0012.jpg")
        unknown_image = cv2.cvtColor(unknown_image, cv2.COLOR_BGR2RGB)
        unknown_image = np.array(Image.fromarray(unknown_image))
        unknown_encodings = face_recognition.face_encodings(unknown_image)
        if len(unknown_encodings):
            results = face_recognition.compare_faces(encodings, unknown_encodings[0])
            print(results)
            match_count = results.count(True)
            print('Found', match_count, 'matche(s) !')
            required_matches =  2# @param int
            if match_count >= required_matches:
                cv2.imwrite(os.path.join(out,out_prexix+str(count).zfill(6)+'.jpg'), face)
                # preview = cv2.circle(face, (50, 50), 25, (0, 255, 0), 25)
                # preview = cv2.rectangle(image, (x-padding,y-padding), (x+w+padding,y+h+padding), (0,255,0), 20)
                # cv2.imshow(imshowname, image_resize(preview, height = 800))
                # cv2.waitKey(1)
            else:
                # preview = cv2.rectangle(image, (x-padding,y-padding), (x+w+padding,y+h+padding), (0,0,255), 20)
                # cv2.imshow(imshowname, image_resize(preview, height = 800))
                # cv2.waitKey(1)
                pass
        else:
            print('No encodings found')
            pass

    count += 1
    if limit_frames != -1:
      if count > (start_offset+limit_frames): break
    # if count>50: break

# clean up window when done
# cv2.destroyAllWindows()

cv2.CAP_PROP_POS_FRAMES: 1

 START PROCESSING <VideoCapture 0x7fd1bbf26eb0>
/content/drive/MyDrive/FACE_RECOG_EXTRACT/MEDIA_TO_SEARCH/Daniel Craig James Bond Monologue - SNL.mp4
/content/drive/MyDrive/FACE_RECOG_EXTRACT/OUT/daniel_craig__late_night_show




Process frame:  499 / 10016   --   5 %
>> Found 2 face(s)
------ Face 0
too small (60)
------ Face 1
too small (60)

Process frame:  500 / 10016   --   5 %
>> Found 3 face(s)
------ Face 0
too small (60)
------ Face 1
too small (60)
------ Face 2
too small (60)

Process frame:  501 / 10016   --   5 %
>> Found 4 face(s)
------ Face 0
too small (60)
------ Face 1
too small (60)
------ Face 2
too small (60)
------ Face 3
too small (60)

Process frame:  502 / 10016   --   5 %
>> Found 3 face(s)
------ Face 0
too small (60)
------ Face 1
too small (60)
------ Face 2
too small (60)

Process frame:  503 / 10016   --   5 %
>> Found 2 face(s)
------ Face 0
too small (60)
------ Face 1
too small (60)

Process frame:  504 / 10016   --   5 %
>>

KeyboardInterrupt: ignored

In [16]:
#@title ZIP Result
zip_dir, out_name = os.path.split(out)
zip_path = os.path.join(zip_dir, out_name.replace(' ','')+'.zip')
print('Inflating ', out_name, ' ...', )
cmd = f"zip -r {zip_path} {out}"
os.system(cmd)
print('Find in Drive =>', zip_path)


Inflating  daniel_craig__late_night_show  ...
Find in Drive => /content/drive/MyDrive/FACE_RECOG_EXTRACT/OUT/daniel_craig__late_night_show.zip


In [15]:
#@title The ZIP already lives in your Drive, will be faster to download it there. Alterntively you can download it here as well.
from google.colab import files
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>